<a href="https://colab.research.google.com/github/kokejohh/PMU_Module_2/blob/main/data_pipeline_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown

gdown.download(f'https://drive.google.com/uc?id=1frp1xNRzYCXD5C3ctqzxuJo1_laJ6dXg', output='data.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1frp1xNRzYCXD5C3ctqzxuJo1_laJ6dXg
To: /content/data.zip
100%|██████████| 48.0k/48.0k [00:00<00:00, 54.0MB/s]


'data.zip'

In [ ]:
import zipfile

with zipfile.ZipFile('data.zip', 'r') as zip_ref:
  zip_ref.extractall('.')

In [ ]:
import os
import pandas as pd

main_dir = '42Workshop-main'
df = pd.read_csv(os.path.join(main_dir, 'data', 'raw', 'market.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4362 entries, 0 to 4361
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  1614 non-null   object 
 1   Price     4188 non-null   float64
 2   Rating    2312 non-null   float64
 3   Stock     3010 non-null   object 
 4   Discount  3970 non-null   float64
dtypes: float64(3), object(2)
memory usage: 170.5+ KB


In [ ]:
#ingest
import shutil

def ingest():
  src_dir = os.path.join(main_dir, 'data', 'raw')
  dest_dir = os.path.join(main_dir, 'data', 'staged')

  os.makedirs(os.path.join(main_dir, 'data', 'staged'), exist_ok=True)

  for file in ['market.csv']:
    shutil.copy(os.path.join(src_dir, file), os.path.join(dest_dir, file))
  print('ingest successfully')

ingest()

ingest successfully


In [ ]:
#stage
import numpy as np

def stage():
  src_dir = os.path.join(main_dir, 'data', 'staged')

  market = pd.read_csv(os.path.join(src_dir, 'market.csv'))

  for name, df in [('market', market)]:
    print(f'[STAGE]:{name}:{df.shape[0]} rows, {df.shape[1]} columns')

  # Fill missing categorical columns with 'Unknown' since NaN > 50%
  market['Category'] = market['Category'].fillna('Unknown')

  # Fill missing categorical columns with mode
  market['Stock'] = market['Stock'].fillna(market['Stock'].mode()[0])

  # Fill missing numerical coulumns with median
  market['Price'] = market['Price'].fillna(market['Price'].median())
  market['Rating'] = market['Rating'].fillna(market['Rating'].median())
  market['Discount'] = market['Discount'].fillna(market['Discount'].median())

  # Fill id
  market['row_id'] = np.arange(1, len(df) + 1)

  display(market)

  market.to_csv(os.path.join(src_dir, 'market_clean.csv'), index=False)

stage()

[STAGE]:market:4362 rows, 5 columns


,Category,Price,Rating,Stock,Discount,row_id
0,Unknown,5548.0,1.870322,In Stock,0.0,1
1,Unknown,3045.0,4.757798,In Stock,38.0,2
2,Unknown,4004.0,3.082060,In Stock,0.0,3
3,Unknown,4808.0,1.492085,In Stock,33.0,4
4,Unknown,1817.0,3.082060,Out of Stock,23.0,5
...,...,...,...,...,...,...
4357,Unknown,4436.0,4.728335,In Stock,49.0,4358
4358,B,6236.0,3.082060,Out of Stock,4.0,4359
4359,Unknown,3283.0,3.082060,Out of Stock,9.0,4360
4360,D,2999.0,4.425995,In Stock,40.0,4361


In [ ]:
#transform

def transform():
  src_dir = os.path.join(main_dir, 'data', 'staged')

  market = pd.read_csv(os.path.join(src_dir, 'market_clean.csv'))

  # create new columns
  market['Price_after_discount'] = market['Price'] * (1 - df['Discount'] / 100)
  market['is_in_stock'] = market['Stock'] == 'In Stock'

  # aggregate by category
  summary = market.groupby('Category').agg(
      avg_price = ('Price_after_discount', 'mean'),
      avg_rating = ('Rating', 'mean'),
      # total_stock = ('Stock', lambda x: (x == 'In Stock').count()),
      total_stock = ('is_in_stock', 'sum'),
      num_products=('row_id', 'count')
  ).reset_index()

  os.makedirs(os.path.join(main_dir, 'data', 'output'), exist_ok=True)

  summary.to_csv(os.path.join(main_dir, 'data', 'output', 'retail_summary.csv'))

transform()

In [ ]:
#load
import sqlite3

def load():
  main_dir = '42Workshop-main'
  csv_path = os.path.join(main_dir, 'data', 'output', 'retail_summary.csv')
  db_path = os.path.join(main_dir, 'data', 'output', 'db_summary.db')

  df = pd.read_csv(csv_path)
  conn = sqlite3.connect(db_path)

  df.to_sql('summary', conn, if_exists='replace', index=False)

  conn.close()

  print('load successfully')

load()

load successfully


In [ ]:
# example
# import pandas as pd

# df = pd.read_csv('./42Workshop-main/data/staged/market.csv')
# df.dropna(subset=['Category', 'Stock', 'Discount'], inplace=True)
# df['Rating'] = df['Rating'].fillna(0)
# df = df.fillna(0)
# df

,Category,Price,Rating,Stock,Discount
6,C,667.0,3.668341,In Stock,41.0
7,A,7125.0,4.983998,Out of Stock,7.0
8,A,2777.0,2.678384,In Stock,6.0
11,A,3772.0,4.890750,In Stock,45.0
15,A,7936.0,3.032832,In Stock,44.0
...,...,...,...,...,...
4342,D,9847.0,3.667649,Out of Stock,49.0
4343,B,843.0,2.752754,Out of Stock,48.0
4345,B,7421.0,0.000000,Out of Stock,32.0
4349,A,7779.0,0.000000,In Stock,29.0
